# Improving on cleaning the original Datasets

In [52]:
import pandas as pd
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 70)
import numpy as np
from tqdm import tqdm_notebook as tqdm
import _pickle as pickle

In [53]:
with open("stocks_df.pickle",'rb') as fp:
    stocks_df = pickle.load(fp)

### Setting the Index to the Date

In [54]:
def setting_index(df):
    df['Quarter end'] = pd.to_datetime(df['Quarter end'])
    df.set_index("Quarter end", inplace=True)
    return df

for i in tqdm(stocks_df.keys()):
    stocks_df[i] = setting_index(stocks_df[i])

### Replacing all "None" and 0 values with NaN

In [55]:
for i in tqdm(stocks_df.keys()):
    stocks_df[i].replace(["None", 0], np.nan, inplace=True)

### Converting all values to numeric values

In [56]:
for i in tqdm(stocks_df.keys()):
    cols = stocks_df[i].columns
    for c in cols:
        stocks_df[i][c] = pd.to_numeric(stocks_df[i][c])

### Filling NaN in each DF


In [58]:
fill = {}

for i in tqdm(stocks_df.keys()):
    fill[i] = stocks_df[i].fillna(method='ffill')

## Creating new columns derived from the originals

In [18]:
def price_increase(df, percent=1.00):
    """
    Returns a boolean determining whether the price will increase in the next quarter
    Optional: determining if the price increased by more than a specified percentage
    """
    return (df.Price.shift(1)*percent) > df.Price

In [19]:
def percent_improvement(df, cols):
    """
    Creates a new DataFrame, 
    then shows the percent change from the previous to the current quarter
    """
    pcnt_df = pd.DataFrame(index=df.index)
    
    pcnt_df['price_will_increase?'] = price_increase(df)
    
    for col in cols:
        if df[col].dtype != bool:
            pcnt_df[col+" %-increase"] = (df[col] / df[col].shift(-1)) - 1
    
    return pcnt_df

In [63]:
pcnt_df = {}

for i in tqdm(stocks_df.keys()):
    pcnt_df[i] = percent_improvement(fill[i], fill[i].columns)

### Excluding the first and last rows
This is done because the last row has no data to compare percent improvements to and the first row does not have any data to show if the price will increase in the future.

In [64]:
for i in tqdm(pcnt_df.keys()):
    pcnt_df[i] = pcnt_df[i][1:-1]

### Combining all stock DFs into one

In [75]:
big_df = pd.DataFrame()

for i in tqdm(pcnt_df.keys()):
    big_df = big_df.append(pcnt_df[i], sort=False)

### Dropping all NaN values

In [102]:
main_df = big_df[~big_df.isin([np.nan, np.inf, -np.inf]).any(1)]

In [103]:
main_df

,price_will_increase?,Shares %-increase,Shares split adjusted %-increase,Split factor %-increase,Assets %-increase,Current Assets %-increase,Liabilities %-increase,Current Liabilities %-increase,Shareholders equity %-increase,Non-controlling interest %-increase,Preferred equity %-increase,Goodwill & intangibles %-increase,Long-term debt %-increase,Revenue %-increase,Earnings %-increase,Earnings available for common stockholders %-increase,EPS basic %-increase,EPS diluted %-increase,Dividend per share %-increase,Cash from operating activities %-increase,Cash from investing activities %-increase,Cash from financing activities %-increase,Cash change during period %-increase,Cash at end of period %-increase,Capital expenditures %-increase,Price %-increase,Price high %-increase,Price low %-increase,ROE %-increase,ROA %-increase,Book value of equity per share %-increase,P/B ratio %-increase,P/E ratio %-increase,Cumulative dividends per share %-increase,Dividend payout ratio %-increase,Long-term debt to equity ratio %-increase,Equity to assets ratio %-increase,Net margin %-increase,Asset turnover %-increase,Free cash flow per share %-increase,Current ratio %-increase
Quarter end,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-03-31,True,3.917108e-03,3.917108e-03,0.000000,0.010803,-0.072407,0.011316,-0.109788,0.009697,0.000000,-0.666667,0.000000,0.046062,0.096547,1.797101,1.808824,1.785714,1.689655,0.010638,-0.200413,-0.081037,0.290541,-0.266667,0.102804,-0.087237,0.034317,0.055814,0.009918,0.032086,0.035831,0.045501,0.078341,-0.130088,0.010574,-0.035870,-0.003484,0.038534,0.053241,-0.041667,0.309091,0.042068
2018-12-31,True,1.404392e-03,1.404392e-03,0.000000,0.009496,-0.065244,0.014665,0.041473,-0.003265,0.000000,2.000000,0.000000,0.032178,-0.176914,-0.807799,-0.809524,-0.808219,-0.800000,0.027322,-0.441109,0.045763,-1.528571,-4.750000,0.163043,0.047377,0.065464,0.073211,0.056805,0.167534,0.171756,-0.042588,0.028436,-0.037172,0.010687,-0.150639,0.076483,-0.050233,0.183562,0.000000,-1.486726,-0.102564
2018-09-30,True,1.048238e-03,1.048238e-03,0.000000,0.012012,-0.026706,0.002616,-0.048322,0.036556,0.000000,0.000000,0.000000,0.000131,0.103007,0.495833,0.493724,0.489796,0.494845,0.000000,0.540925,0.109023,11.173913,-1.571429,-0.041667,0.100559,0.077413,0.079347,0.075259,0.099542,0.100840,0.036388,0.055000,0.000404,0.010570,-0.090286,-0.035779,0.025009,0.111675,0.000000,10.300000,0.022848
2018-06-30,True,1.584925e-03,1.584925e-03,0.000000,0.021473,0.075303,0.021596,-0.189537,0.021577,0.000000,0.000000,0.000000,0.125185,-0.013880,0.568627,0.582781,0.580645,0.564516,0.000000,1.178295,-0.108878,-1.063889,-0.666667,0.078652,-0.091371,0.048143,0.037608,0.060127,0.072393,0.067265,0.020282,0.005025,-0.041085,0.010683,-0.064064,0.100880,0.000368,0.076503,0.000000,-1.072993,0.326574
2018-03-31,True,4.199764e-03,4.199764e-03,0.000000,0.005165,-0.027916,0.004726,0.137755,0.006403,0.000000,-0.666667,0.000000,-0.081954,0.130528,-3.550000,-3.516667,-3.583333,-3.583333,-0.005435,-0.440347,-0.037097,1.250000,20.000000,7.900000,-0.046774,-0.094974,-0.090307,-0.100225,0.086667,0.082524,0.044524,-0.033981,0.060855,0.010798,-0.075684,-0.124702,0.043378,0.080283,0.000000,1.075758,-0.145541
2017-12-31,False,0.000000e+00,0.000000e+00,0.000000,0.012804,0.019608,0.026972,0.139093,-0.021920,0.000000,2.000000,0.000000,0.064721,-0.186303,-1.208333,-1.208333,-1.201681,-1.203390,0.045455,-0.408974,0.178707,-1.627451,-2.000000,0.111111,0.176471,0.070954,0.065342,0.077139,-0.151584,-0.159184,-0.061974,0.040404,0.215392,0.010916,0.189228,0.134860,-0.073613,-0.155533,0.000000,-1.634615,-0.104962
2017-09-30,True,0.000000e+00,0.000000e+00,0.000000,0.014374,0.006365,0.009803,-0.066546,0.026125,0.000000,0.000000,0.000000,0.014807,0.120286,0.492228,0.492228,0.506329,0.493671,0.000000,0.466165,0.011538,24.500000,-1.500000,-0.100000,0.019342,0.031549,0.064674,-0.003351,-0.116883,-0.131206,0.026625,0.020619,-0.033333,0.010552,0.141542,-0.011520,0.012239,-0.097210,-0.040000,16.3333

### Multiplying the values in the DF by 100 and rounding to two decimal places

In [104]:
for i in tqdm(main_df.columns[1:]):
    main_df[i] = main_df[i].apply(lambda x: x*100).round(2)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [122]:
main_df['price_will_increase?'].value_counts()

True     2659
False    1701
Name: price_will_increase?, dtype: int64